<a href="https://colab.research.google.com/github/zaeyonz/Wine_Classification/blob/main/Wine_Classification_(Cross_validation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Validation Set***
*   train set를 test set이외에 또 나눔 (20%~30%정도)
*   test set를 사용하지 않고 과대,과소적합인지를 측정하는 가장 간단한 방법


In [1]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()     # class 열을 타깃으로 사용, 나머지 열은 특성 배열에 저장
target = wine['class'].to_numpy()

In [2]:
# train set & test set를 나눔
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state=42)

In [3]:
# train_input과 train_target을 다시 훈련 세트 sub_input, sub_target과 val_input, val_target으로 나눔

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state=42)

In [4]:
# train set와 validation set 의 크기 확인

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


*   ***원래 5197개였던 train set가 4157개로 줄고, validation set는 1040개가 되었음***

# ***Model Training***
*   sub_input, sub_target, val_input, val_target을 사용해 model을 만들고 평가

In [5]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


*   ***확실히 train set에 과대적합되어 있음***
*   매개변수를 바꿔가며 더 좋은 모델을 찾아야함

# ***Cross Validation (교차 검증)***
*   교차 검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있음
*   검증 세트를 떼어 내어 평가하는 과정을 여러번 반복함
*   보통 5-폴드, 10-폴드 교차 검증을 많이 사용함 --> 이렇게 하면 데이터의 80 ~ 90%까지 훈련에 사용 가능
*   ***scikit-learn에는 cross_validation()이라는 교차 검증 함수가 있음***

In [6]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00878143, 0.00689602, 0.00679922, 0.0069108 , 0.00724483]), 'score_time': array([0.00096703, 0.000772  , 0.0008316 , 0.00080395, 0.00094604]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


*   fit_time: 각각의 모델을 훈련하는 시간
*   score_time: 모델을 검증 하는 시간
*   test_score: 점수

In [7]:
# 교차 검증의 최종 점수는 test_score키에 담긴 5개릐 점수를 평균하여 얻음

import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


*   ***cross_validate()는 train_set를 섞어 폴드를 나누지 않음!!***
*   앞서 train_test_split() 함수로 자동으로 전체 데이터를 섞은 후 데이터를 나눴기에 따로 섞을 필요는 없음
*   하지만 만약 교차 검증을 할 때 train set를 섞으려면 ***분할기(splitter)***를 지정해야함
---

*   cross_validate() 함수는 기본적으로 회귀 모델의 경우 KFold 분할기를 사용함
*   분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용함

In [8]:
# 앞서 수행한 교차 검증은 아래와 동일함!!

from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
# 만약 train set를 섞은 후 10-폴드 교차 검증을 수행하려면..

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)      # n_splits 매개변수는 몇(k) 폴드 교차 검증을 할지 정함
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# ***하이퍼파라미터 튜닝***
*   머신러닝 모델이 학습하는 파라미터를 모델 파라미터라고 부름
*   반면, ***모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 함***
*   먼저 라이브러리가 제공하는 기본값을 그대로 사용 후, 매개변수를 바꿔가며 모델을 훈련, 교차 검증 함
---
*   한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾으면 안됨!
*   ***여러 매개변수를 동시에 바꿔가며 최적의 값을 찾아야함!!***

# ***Grid Search (그리드 서치)***
*   많은 매개변수들을 한번에 최적의 값을 찾아주는 클래스
*   ***하이퍼파라미터 탐색과 교차 검증을 한 번에 수행함!!***
*   scikit-learn의 ***GridSearchCV()***

In [10]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}         # 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듦

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)    # n_jobs 매개변수에서 병렬시행에 사용할 CPU코어 수를 지정, 기본값 1, -1로 지정하면 모든 코어를 사용

*   GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만들었음
*   결정트리 클래스의 객체를 생성하자마자 바로 전달함
*   그다음 일반 모델을 훈련하는 것처럼 gs객체에 fit() 메서드 호출

In [11]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

*   그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 시행함
*   GridSearchCV의 cv 매개변수 기본값은 5임.
*   min_impurity_decrease 값마다 5-폴드 교차 검증을 수행 --> ***결국 5 * 5=25개의 모델을 훈련***
---
*   교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 train set로 모델을 다시 만들어야함!!
*   ***scikit-learn에선 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 train set에 자동으로 모델을 훈련함!***
*   최적의 모델은 gs 객체의 best_estimator_ 속성에 저장되어 있음

In [12]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [13]:
# 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있음

print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [14]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score'키에 저장되어 있음

print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


*   첫 번째 값이 가장 큼 --> 0.0001!!
*   수동으로 고르는 것보다 ***numpy에서 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출 가능***
*   ***그다음 이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있음*** --> ***최상의 검증점수를 만든 매개변수 조합임***

In [15]:
# 앞서 출력한 gs.best_params_ 와 동일한지 확인

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


# ***더 복잡한 매개변수 조합 탐색***
*   min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정
*   여기에다가 ***max_depth로 트리의 깊이를 제한***, ***min_samples_split으로 노드를 나누기 위한 최소 샘플수도 골라봄***

In [16]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),        # 1번
          'max_depth': range(5, 20, 1),                                     # 2번
          'min_samples_split': range(2, 100, 10)                            # 3번
          }



1.   numpy arange() 함수는 첫번째 매개변수 값에서 시작하는 두번째 매개변수에 도달할 때까지 세번째 매개변수를 계속 더한 배열을 만듦 ---> (0.0001 , 0.0002, .... , 0.0009, 0.001)
2.   파이썬 range() 함수는 비슷하지만 ***정수만 사용가능!*** ---> (5, 6, ... ,19, 20)
3.   2번과 동일  ---> (2, 12, 22, 32, ... ,82, 92)
----

*   따라서 위 매개변수로 수행할 교차 검증 횟수는 9 * 15 * 10 = 1350 개
*   기본 5-폴드 교자 검증을 수행하므로, ***1350 X 5 = 6750개***

In [17]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [18]:
# 최상의 매개변수 조합 확인

print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [19]:
# 최상의 교차 검증 점수 확인

print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# ***Random Search (랜덤 서치)***
*   매개변수릐 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있음.
*   또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있음
*   이럴 때 ***Random Search (랜덤 서치)*** 를 사용!
---
*   Random Search에는 매개변수 값의 목록을 전달하는 것이 아닌, ***매개변수를 샘플링할 수 있는 확률 분포 객체를 전달***

In [20]:
# 싸이파이에서 2개의 확률 분포 클래스를 임포트
from scipy.stats import uniform, randint

*   uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑음
*   이를 ***'균등 분포에서 샘플링 한다'*** 라고 말함
*   ***randint는 정수값, uniform은 실숫값을 뽑음***

# ***randint, uniform 클래스 사용법***

In [21]:
# randint 사용법

rgen = randint(0, 10)
rgen.rvs(10)        # rvs() = 랜덤 표본 생성 (random variable sampling)

array([6, 9, 0, 9, 1, 2, 1, 4, 1, 8])

In [22]:
# 10개라 고르지 못하니 1000개를 샘플링 후 각 숫자의 개수 확인
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 91, 102,  93,  87, 103, 109, 108, 112,  95, 100]))

*   개수가 늘어나니 0에서 9까지의 숫자가 어느 정도 고르게 추출된 것을 볼 수 있음

In [23]:
# uniform 사용법

ugen = uniform(0, 1)    # 0 에서 1사이의 10개의 실수 추출
ugen.rvs(10)

array([0.35121309, 0.78409649, 0.82538874, 0.11721481, 0.99517556,
       0.58163792, 0.00445814, 0.91542491, 0.28923077, 0.71244148])

# ***randint와 uniform을 사용하여 매개변수 조합 찾기***
*   min_samples_leaf 매개변수를 탐색 대상에 추가
*   min_samples_leaf는 리프노드가 되기 위한 최소 샘플의 개수
*   어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않음


In [26]:
params = {'min_impurity_decrease': uniform(0.0001,0.001),       # 0.0001에서 0.001 사이의 실숫값을 샘플링
          'max_depth': randint(20,50),                      # 20에서 50 사이의 정수 샘플링
          'min_samples_split': randint(2,25),               # 2에서 25 사이의 정수 샘플링
          'min_samples_leaf': randint(1, 25)                # 1에서 25 사이의 정수 샘플링
          }

*   ***샘플링 횟수는 scikit-learn의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정함***

In [29]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)        # params에 정의된 매개변수 범위에서 총 100번을 샘플링 , 그리드 서치 보다 훨씬 적은 횟수의 교차 검증 시행
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a555e781c30>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7a555e7816c0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a555e781720>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a555e782e60>},
                   random_state=42)

In [32]:
# 최적의 매개변수값 확인

print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [35]:
# 최고 교차 검증 점수 확인 (= validation set 점수)

print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [36]:
# 최적의 모델은 이미 전체 훈련 세트로 훈련되어 best_estimator_ 속성에 저장되어 있음 --> 이 모델이 최종 모델, 훈련!

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


*   test set 점수는 validation set에 대한 점수보다 조금 작은 것이 일반적